In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    export_model_path: Path

In [3]:
from objectDetection.constants import *
from objectDetection.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pt",
            training_data="dataset.yaml",
            export_model_path="torchserve/models/detection.onnx",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [5]:
from ultralytics import YOLO

In [6]:
import shutil

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    @staticmethod
    def load_model(path: Path) -> YOLO:
        return YOLO(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.results =  self.model.val(data=self.config.training_data, imgsz=self.config.params_image_size,batch=self.config.params_batch_size)

    def export_model(self):
        self.model.export(format="onnx")
        shutil.copy("artifacts/training/model.onnx", self.config.export_model_path)

    def save_score(self):
        save_json(path=Path("scores.json"), data=self.results.results_dict)


In [9]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.export_model()

except Exception as e:
   raise e

[2024-02-04 11:52:00,835: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-04 11:52:00,842: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-04 11:52:00,844: INFO: common: created directory at: artifacts]
Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.2.0+cu121 CPU (12th Gen Intel Core(TM) i5-1235U)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/ajith/projects/Multivideo_Objectdetection_MLOPS_Project/artifacts/data_ingestion/dataset/valid/labels.cache... 90 images, 0 backgrounds, 0 corrupt: 100%|██████████| 90/90 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.21s/it]


                   all         90       1502      0.728     0.0763      0.107     0.0527
               bicycle         90         30          1          0   0.000118   5.89e-05
                   bus         90          4          1          0    0.00417    0.00207
                   car         90       1030      0.642      0.382      0.459       0.21
             motorbike         90        400          0          0    0.00646    0.00208
                person         90         38          1          0     0.0636     0.0487
Speed: 0.6ms preprocess, 46.0ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val3
[2024-02-04 11:52:13,325: INFO: common: json file saved at: scores.json]
Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.2.0+cu121 CPU (12th Gen Intel Core(TM) i5-1235U)

PyTorch: starting from 'artifacts/training/model.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 9, 2100) (5.9 MB)

ONNX: starting export with onnx 1.15.0 opse